In [83]:
from qiskit.circuit import QuantumCircuit, Parameter
import numpy as np
phi = Parameter("1")
phi2 = Parameter("2")
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.rx(phi,qubit=[0])
circuit.rx(phi2,qubit=[1])
print(circuit)
print(circuit.parameters)
value = np.random.random()
parameters = [value for _ in range(2)]
print(circuit.bind_parameters(parameters))

       ┌───┐┌───────┐
q68_0: ┤ H ├┤ Rx(1) ├
       ├───┤├───────┤
q68_1: ┤ H ├┤ Rx(2) ├
       ├───┤└───────┘
q68_2: ┤ H ├─────────
       ├───┤         
q68_3: ┤ H ├─────────
       └───┘         
ParameterView([Parameter(1), Parameter(2)])
       ┌───┐┌───────────────────────┐
q68_0: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤├───────────────────────┤
q68_1: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤└───────────────────────┘
q68_2: ┤ H ├─────────────────────────
       ├───┤                         
q68_3: ┤ H ├─────────────────────────
       └───┘                         


In [23]:
individual = "0000000"
elitism_population = ["1110111","1111111"]
match_percentage = []
def match(subject_1,subject_2):
    return sum([int(subject_1[i] == subject_2[i]) for i in range(0,len(subject_1))])/len(subject_1)
for elite in elitism_population:
    match_percentage.append(match(individual,elite))
    print(match_percentage)
print(sum(match_percentage)/len(match_percentage))

[0.14285714285714285]
[0.14285714285714285, 0.0]
0.07142857142857142


In [14]:
from Experiments import saveLoad
from CLQGA import Individual
output = saveLoad("load","experiment_18_april_mucBoost_1", None)
experiment_population = output[0]
experiment_duration = output[1]
experiment_average_fitness_50_increment = output[2]
experiment_average_crowd_score_50_increment = output[3]

In [15]:
# [population,time.gmtime((time.perf_counter() - start_total_run_time)),data_average_fitness, data_average_crowd_score]
from HelperCLQGA import genome_to_circuit
print(output[0])
print(genome_to_circuit(output[0][0].chromosome, 4, 20)[0])

[<CLQGA.Individual object at 0x7fb42337bf10>, <CLQGA.Individual object at 0x7fb4241337d0>, <CLQGA.Individual object at 0x7fb4233e8c50>, <CLQGA.Individual object at 0x7fb423989810>, <CLQGA.Individual object at 0x7fb424110290>, <CLQGA.Individual object at 0x7fb424112050>, <CLQGA.Individual object at 0x7fb4234ff090>, <CLQGA.Individual object at 0x7fb4234fd150>, <CLQGA.Individual object at 0x7fb4234fde90>, <CLQGA.Individual object at 0x7fb4234fe050>, <CLQGA.Individual object at 0x7fb4234fcf50>, <CLQGA.Individual object at 0x7fb4234fd910>, <CLQGA.Individual object at 0x7fb4234fe6d0>, <CLQGA.Individual object at 0x7fb4234ffdd0>, <CLQGA.Individual object at 0x7fb4234fc8d0>, <CLQGA.Individual object at 0x7fb4234ff490>, <CLQGA.Individual object at 0x7fb4234ff0d0>, <CLQGA.Individual object at 0x7fb422e703d0>, <CLQGA.Individual object at 0x7fb422e72350>, <CLQGA.Individual object at 0x7fb422e72550>, <CLQGA.Individual object at 0x7fb422e70bd0>, <CLQGA.Individual object at 0x7fb4233a03d0>, <CLQGA.In